In [1]:
# import des librairies
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import numpy as np 

In [2]:
# import des données
df = pd.read_csv("../data/silver/silver_data.csv")
df.head(3)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"[""Analyser les résultats d'un projet"", ""Déclin...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,['Contrôler et faire appliquer le respect de d...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13


In [3]:
df.isnull().sum()

jobs                      0
description               0
loc                     902
datePosted              902
workhours              1542
salary                 1591
contract_type             0
experience                0
diploma                1827
skills                  861
qualifications         1072
industry               1031
company                 965
size                    706
company_description     947
tools                   930
ID_dep                    0
ville                   228
date                      0
dtype: int64

### preprocessing

In [4]:
df = df[df['company_description'].notna()]
df.shape

(954, 19)

In [9]:
company_description = df["company_description"].tolist()
texts = company_description[:20]
texts[0]

"Pure player en Data et Intelligence Artificielle, Quantmetry accompagne des grands comptes de tous secteurs (PSA, BPCE, Crédit Agricole, Safran, Orange, EDF, SNCF, Decathlon...) à valoriser leur patrimoine de données et à mettre en ?uvre des solutions d'IA. Pour ce faire, Quantmetry s'appuie sur ses expertises en Conseil Stratégique et Opérationnel, Data Science, Architecture & Data Engineering."

### Models

In [6]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [7]:
def summarize_texts(texts, model, tokenizer):
    """
    Summarizes a list of texts using the provided model and tokenizer.
    """
    # Prefix each text with "summarize: " and tokenize them
    texts = ["summarize: " + text for text in texts]
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids

    # Generate the summarization output
    outputs = model.generate(
        inputs, 
        max_length=150, 
        min_length=40, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True)

    # Decode each summary output
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return summaries

In [10]:
summaries = summarize_texts(texts, model, tokenizer)

In [13]:
summaries[:3]

["Quantmetry accompagne des grands comptes de tous les secteurs (PSA, BPCE, Crédit Agricole, Safran, Orange, EDF, SNCF, Decathlon...). pour ce faire, Quantmetry s'appuie sur ses expertises en Conseil Stratégique et Opérationnel, Data Science, Architecture & Data Engineering.",
 "Quantmetry accompagne des grands comptes de tous les secteurs (PSA, BPCE, Crédit Agricole, Safran, Orange, EDF, SNCF, Decathlon...). pour ce faire, Quantmetry s'appuie sur ses expertises en Conseil Stratégique et Opérationnel, Data Science, Architecture & Data Engineering.",
 "Nexity est aujourd'hui leader sur les différents métiers de l immobilier tel que la promotion, services gestion d actifs et Régénération urbaine. Nexity est reconnu pour sa capacité à apporter une réponse globale aux besoins de ses clients : particuliers, entreprises et collectivités territoriales."]